In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
import numpy as np
from pathlib import Path

custom_params = {"axes.spines.right": False, "axes.spines.top": False}

sns.set_theme(
    context="notebook",
    style="ticks", 
    palette="colorblind",
    font_scale=1.1,
    rc=custom_params
)

# For SVG output
plt.rcParams['svg.fonttype'] = 'none'
figdir = Path("../figures/svg")

basedir = Path("../analysis/g4_overlap")


## Fisher

In [2]:
dataf = []
for tsv in (basedir / "fisher").rglob("*.tsv"):
    
    strat = tsv.stem
    d = pd.read_csv(tsv, sep="\t", comment="#", header=0)
    d["Stratification"] = strat

    with open(tsv) as f:
        for line in f:
            if line.startswith("# Number"):
                stat, number = line.strip().split(": ")
                stat = stat[2:].replace(" ", "_")
                d[stat] = int(number)
    dataf.append(d)

dataf = pd.concat(dataf).reset_index(drop=True)
#dataf = dataf.query("Stratification.str.startswith('SimpleRepeat') | Stratification.str.startswith('gc')").copy()
dataf["pct_overlaps"] = dataf["Number_of_overlaps"] / dataf["Number_of_db_intervals"]

# From: https://groups.google.com/g/bedtools-discuss/c/wYTmdKnkN54/m/7RTOhoA8BQAJ
# - "right": low right tail means more overlap that expected
# - "ratio": ~enrichment

# Bonferroni correction for multiple testing
dataf["padj"] = 100 * dataf["right"] * len(dataf) 
dataf["padj"] = dataf["padj"].apply(lambda x: min(x, 1))

dataf.sort_values("ratio", ascending=False).head(20)

,left,right,two-tail,ratio,Stratification,Number_of_query_intervals,Number_of_db_intervals,Number_of_overlaps,Number_of_possible_intervals_(estimated),pct_overlaps,padj
68,1.000000e+00,3.145500e-94,3.145500e-94,1009.766,SimpleRepeat_homopolymer_ge21_GC_slop5,1352359,66,64,44024303,0.969697,5.944995e-90
59,1.000000e+00,0.000000e+00,0.000000e+00,213.356,SimpleRepeat_imperfecthomopolge21_GC_slop5,1352359,3439,2993,44253247,0.870311,0.000000e+00
120,1.000000e+00,0.000000e+00,0.000000e+00,144.985,SimpleRepeat_homopolymer_ge12_GC_slop5,1352359,3398,2673,54428649,0.786639,0.000000e+00
94,1.000000e+00,0.000000e+00,0.000000e+00,106.191,SimpleRepeat_imperfecthomopolge11_GC_slop5,1352359,107713,77378,54461223,0.718372,0.000000e+00
91,1.000000e+00,0.000000e+00,0.000000e+00,18.978,SimpleRepeat_homopolymer_7to11_GC_slop5,1352359,101792,30204,60895954,0.296723,0.000000e+00
66,1.000000e+00,0.000000e+00,0.000000e+00,3.688,SimpleRepeat_homopolymer_4to6_GC_slop5,1352359,12048931,613434,63585273,0.050912,0.000000e+00
163,1.000000e+00,0.000000e+00,0.000000e+00,1.833,SimpleRepeat_imperfecthomopolge11_slop5,1352359,1678630,76546,51891858,0.045600,0.000000e+00
18,1.000000e+00,8.212100e-41,1.597500e-40,1.047,AllHomopolymers_ge7bp_imperfectge11bp_slop5,1352359,3819656,95925,56119852,0.025114,1.552087e-36
155,4.420700e-235,1.000000e+00,8.802500e-235,0.801,AllTandemRepeats_le50bp_slop5,1352359,929746,21142,48097672,0.022740,1.000000e+00
36,8.073300e-47,1.000000e+00,1.669900e-46,0.740,SimpleRepeat_triTR_14to49_slop5,1352359,103898,2104,49806944,0.020251,1.000000e+00


In [3]:
d = dataf.copy()
print(d.query("padj < 0.01").sort_values("ratio", ascending=False).to_csv(index=False))

left,right,two-tail,ratio,Stratification,Number_of_query_intervals,Number_of_db_intervals,Number_of_overlaps,Number_of_possible_intervals_(estimated),pct_overlaps,padj
1.0,3.1455e-94,3.1455e-94,1009.766,SimpleRepeat_homopolymer_ge21_GC_slop5,1352359,66,64,44024303,0.9696969696969697,5.944995e-90
1.0,0.0,0.0,213.356,SimpleRepeat_imperfecthomopolge21_GC_slop5,1352359,3439,2993,44253247,0.8703111369584181,0.0
1.0,0.0,0.0,144.985,SimpleRepeat_homopolymer_ge12_GC_slop5,1352359,3398,2673,54428649,0.7866391995291347,0.0
1.0,0.0,0.0,106.191,SimpleRepeat_imperfecthomopolge11_GC_slop5,1352359,107713,77378,54461223,0.7183719699572011,0.0
1.0,0.0,0.0,18.978,SimpleRepeat_homopolymer_7to11_GC_slop5,1352359,101792,30204,60895954,0.29672272870166616,0.0
1.0,0.0,0.0,3.688,SimpleRepeat_homopolymer_4to6_GC_slop5,1352359,12048931,613434,63585273,0.05091190247499965,0.0
1.0,0.0,0.0,1.833,SimpleRepeat_imperfecthomopolge11_slop5,1352359,1678630,76546,51891858,0.045600281181677915,0.0
1.0,8.212100000000001e-4